In [21]:
import os
import sys
import importlib

import pandas as pd
import numpy as np
import sklearn.tree as tree
import sklearn.model_selection as sel
import sklearn.metrics as metrics
import sklearn.ensemble as ensemble
import seaborn as sb

from sklearn import svm


# add project modules to the path
path_to_module = os.path.abspath(os.path.join(os.getcwd(),"..","src/"))
sys.path.append(path_to_module)

import src.models.train_model as train

I'm now repeating code a lot here, bad! Will need to write bits of this

In [2]:
path_to_data = os.path.abspath(os.path.join(os.getcwd(),
                                            "..",
                                            "data/processed/"
                                           ))

data_dict = train.load_processed_data(file_path=path_to_data)

loading files from /home/edd/Documents/Projects/minst-ensemble/data/processed: 100%|██████████| 4/4 [00:01<00:00,  2.10it/s]


In [3]:
X = data_dict["X_train"]
y = data_dict["y_train"]

In [4]:
X_train, X_test, y_train, y_test = sel.train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=y)

In [5]:
X_train_smol, X_test_smol, y_train_smol, y_test_smol = sel.train_test_split(X,
                                                    y,
                                                    test_size=0.09,
                                                    train_size=0.21,
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=y)

In [47]:
ada_clf = ensemble.AdaBoostClassifier(tree.DecisionTreeClassifier(class_weight="balanced",),
                                      
                                     random_state=42,
                                     )

In [48]:
ada_params = {"n_estimators": [100,200,500,1000,2000],
              "learning_rate": [0.01, 0.1, 1, 10]
             }
            

In [49]:
grid_search = sel.GridSearchCV(estimator=ada_clf, 
                               param_grid=ada_params,
                               scoring = "f1_macro",
                               n_jobs = 2,
                               cv=3,
                               verbose = 10
                              )

In [50]:
grid_search.fit(X_train_smol, y_train_smol)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    4.6s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    9.5s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:   23.7s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:   34.0s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:   55.5s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:  1.2min
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:  1.6min
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  1.9min
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed:  2.4min
[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:  2.5min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                          base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                                                criterion='gini',
                                                                                max_depth=None,
                                                                                max_features=None,
                                                                                max_leaf_nodes=None,
                                                                                min_impurity_decrease=0.0,
                                                                                min_impurity_split=None,
                                                                                min_samples_leaf=1,
                                                                          

In [51]:

grid_search.best_params_

{'learning_rate': 0.01, 'n_estimators': 100}

In [52]:
grid_search.best_score_

0.7085986138018274

not really very promising...